In [6]:
#from qiskit.providers.basicaer import QasmSimulatorPy  # local simulator
import numpy as np
# Qiskit imports
from qiskit import BasicAer
from qiskit.algorithms import QAOA
from qiskit.tools.jupyter import *
from qiskit.providers.aer import QasmSimulator
from qiskit.algorithms.optimizers import COBYLA
from qiskit.providers.basicaer import QasmSimulatorPy  # local simulator
from qiskit_optimization.algorithms import CplexOptimizer,MinimumEigenOptimizer
from qiskit.utils.algorithm_globals import algorithm_globals
from qiskit.utils import QuantumInstance
from qiskit_optimization.converters import *
import random
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
import copy
# Problem modelling imports
from docplex.mp.model import Model
from qiskit_optimization.translators import from_docplex_mp
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.problems.variable import VarType
from objective import tsp_objective_ws
def wsqaoa(N,qubo,H):
    algorithm_globals.random_seed = 123

    #check if qubo is convex or not then adapt appropriately
    def relax_problem(problem) -> QuadraticProgram:
        """Change all variables to continuous."""
        relaxed_problem = copy.deepcopy(problem)
        for variable in relaxed_problem.variables:
            variable.vartype = VarType.CONTINUOUS
        return relaxed_problem
    def get_Qc(H,N):
        varlist=[]
        for i in range(0,N):
            for m in range(0,N):
                varlist.append('x_'+str(i)+'_'+str(m))
        nh=str(H)
        nh=nh.replace('-','+-')
        lish=nh.split('+')
        Q=np.zeros([len(varlist),len(varlist)])
        c=np.zeros(len(varlist))
        for i in lish:
            if '^2' in i:
                for j in range(0,len(varlist)):
                    if varlist[j] in i:
                        i=i.replace(varlist[j]+'^2','')
                        Q[j,j]=Q[j,j]+float(i)
            elif '*' in i:
                ls=[]
                for j in range(0,len(varlist)):
                    if varlist[j] in i:
                        ls.append(j)
                        i=i.replace(varlist[j],'')
                i=i.replace('*','')
                Q[ls[0],ls[1]]=Q[ls[0],ls[1]]+float(i) 
                Q[ls[1],ls[0]]=Q[ls[1],ls[0]]+float(i) 
            else:
                for j in range(0,len(varlist)):
                    if varlist[j] in i:
                        i=i.replace(varlist[j],'')
                        c[j]=c[j]+float(i)
        return Q,c,varlist
    def get_simple_convex_qp(Q, c,varlist):
        mdl = Model()
        n_qubits = Q.shape[0]
        x = [mdl.binary_var(i) for i in varlist]
        eigvals = np.linalg.eigvalsh(Q)
        u = eigvals[0]*np.ones(n_qubits)
        objective = mdl.sum([(c[i]) * x[i] for i in range(n_qubits)])
        objective += mdl.sum([(u[i]) * x[i] for i in range(n_qubits)])
        objective += mdl.sum([Q[i, j] * x[i] * x[j] for j in range(n_qubits) for i in range(n_qubits)])
        objective -= mdl.sum([u[i] * x[i] * x[i] for i in range(n_qubits)])
        mdl.minimize(objective)
        qp = from_docplex_mp(mdl)
        return qp

    def reachable_cstar(c_star: float, epsilon: float):
        return max(min(c_star, 1 - c_star), epsilon)
    
    
    optimizer = COBYLA(maxiter=1000, tol=0.0001)
    algorithm_globals.random_seed = 123
    quantum_instance = QuantumInstance(
        BasicAer.get_backend('qasm_simulator'),
        seed_simulator=123,
        seed_transpiler=123,
    )
    
    
    qp = relax_problem(QuadraticProgramToQubo().convert(qubo))
    sol = CplexOptimizer().solve(qp)
    c_stars = sol.samples[0].x
    c_test=np.array(c_stars)
    is_all_zero = np.all((c_test == 0.0))
    
    if is_all_zero==False:
        thetas = [2 * np.arcsin(np.sqrt(c_star)) for c_star in c_stars]
        method='first'

    else:        
        method='second'
        # precision of SDP solver is around 1e-7
        # this term is therefore for error correction
        EPSILON = 1e-6
        Q,c,varlist=get_Qc(H,N)
        qp = get_simple_convex_qp(Q, c,varlist)
        for var in qp.variables:
            var.vartype = VarType.CONTINUOUS
        sol = CplexOptimizer().solve(qp)
        c_stars = sol.samples[0].x



    return c_stars




In [7]:
import os
import numpy as np
arr = os.listdir('instances')
arr=[i for i in arr if '.txt' in i]
for i in arr:
    dmat=np.loadtxt('instances/'+str(i),delimiter=',')
    qubo,H=tsp_objective_ws(dmat)
    cst=wsqaoa(4,qubo,H)
    print(cst)
    #np.loadtxt('instances/'+ins+'.txt',  delimiter=',') 

[0.23357865567694255, 0.23357865733616803, 0.23357865577274314, 0.2335786541135228, 0.18738052453508008, 0.1873805251266648, 0.1873805255475839, 0.18738052495599866, 0.22051381648942978, 0.22051381472703196, 0.22051381529487757, 0.22051381705727402, 0.22197301306558967, 0.22197301257717708, 0.2219730131518373, 0.22197301364024627]
[0.23632621277706556, 0.23632621310520302, 0.23632621244743615, 0.2363262121192933, 0.2206554255198446, 0.22065542580896508, 0.22065542642041605, 0.2206554261312954, 0.23807958877201482, 0.23807958842036092, 0.23807958843800445, 0.23807958878966431, 0.19749122405400257, 0.19749122378839878, 0.19749122381707104, 0.19749122408267478]
[0.23647309942037464, 0.23647309956422397, 0.23647309804422453, 0.23647309790037618, 0.2070046427053683, 0.20700464229919438, 0.207004641657838, 0.20700464206401242, 0.2111342393249726, 0.2111342400669155, 0.21113424167864292, 0.21113424093669914, 0.2318956487438313, 0.23189564826421327, 0.23189564881384203, 0.23189564929345904]
Er

C:\Users\Aubrey\anaconda3\lib\site-packages\qiskit_optimization\algorithms\cplex_optimizer.py:146: UserWarning: CPLEX cannot solve the model
  warn("CPLEX cannot solve the model")


[0.16927044005241582, 0.16927044096963276, 0.16927044005237063, 0.16927044096955096, 0.15887535274798836, 0.15887535411082804, 0.15887535274805253, 0.15887535411085107, 0.17595184020912688, 0.1759518385666637, 0.17595184020915364, 0.17595183856672675, 0.16546773098095946, 0.16546773052554353, 0.1654677309809136, 0.16546773052553887]
Error: Model has non-convex objective: 78.859 x_0_0^2 + 78.859 x_0_0 * x_0_1 + 78.859 x
[0.14877520213181247, 0.1487751979034219, 0.14877520213181483, 0.14877519790351995, 0.18215293213883083, 0.18215293993142373, 0.1821529321388246, 0.18215293993139264, 0.16504337720869489, 0.16504337139363623, 0.16504337720869675, 0.16504337139360253, 0.1420715415241599, 0.14207154197473407, 0.14207154152416196, 0.1420715419747014]
Error: Model has non-convex objective: 72.835 x_0_0^2 + 72.835 x_0_0 * x_0_1 + 72.835 x
[0.16165863733020008, 0.16165863454041468, 0.16165863733019492, 0.16165863454064328, 0.1819901236147664, 0.18199013452621055, 0.18199012361476707, 0.1819901